In [7]:
import requests
from bs4 import BeautifulSoup
import random
import pandas as pd

In [3]:
title = 'Dados'
location = 'Brasil'

work_model = {'1': 'Presencial', '2': 'Remoto', '3': 'Híbrido'}

In [4]:
job_ids = set()
job_work_model = {}

for n_results in range(0,21,10):
    id_WM = random.choice(list(work_model.keys()))
    list_url = f'https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords={title}&location=Brasil&geoId=106057199&f_WT={\
        id_WM}&position=1&start={n_results}'

    response = requests.get(list_url)

    list_data = response.text
    list_soup = BeautifulSoup(list_data, 'html.parser')
    job_cards = list_soup.find_all('div', class_='base-card')

    for card in job_cards:
        if 'data-entity-urn' in card.attrs:
            job_id = card['data-entity-urn'].split(':')[3]
            job_ids.add(job_id)
            job_work_model[job_id] = work_model[id_WM]

job_ids = list(job_ids)

In [5]:
len(job_ids)
print(job_ids)
print(len(job_ids))

['4184262596', '4191196959', '4152258042', '4179429367', '4178256022', '4184711760', '4186563017', '4192442079', '4159081209', '4146772081', '4184358084', '4185677926', '4188601092', '4189220057', '4190106005', '4182151010', '4185696461', '4075025076', '4183504583', '4189742871', '4184709447', '4142098937', '4176740814', '4170063595', '4167869097', '4181438889', '4187579023', '4188230342', '4153856219']
29


In [8]:
job_list = []

for job_id in job_ids:
    job_url = f'https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{job_id}?_l=pt_BR'
    job_response = requests.get(job_url)
    # print(job_response.status_code)

    if job_response.status_code == 200:
        job_post = {}
        job_soup = BeautifulSoup(job_response.text, 'html.parser')

        job_post['job_id'] = job_id

        try:
            job_post['job_title'] = job_soup.find(
                'h2', {'class': 'top-card-layout__title'}
            ).text.strip()
        except:
            job_post['job_title'] = None

        try:
            job_post['company_name'] = job_soup.find(
                'a', {'class': 'topcard__org-name-link'}
            ).text.strip()
        except:
            job_post['company_name'] = None

        job_post['work_model'] = job_work_model.get(job_id, None)

        try:
            job_post['location'] = job_soup.find(
                'span', {'class': 'topcard__flavor topcard__flavor--bullet'}
            ).text.strip()
        except:
            job_post['location'] = None   

        try:
            job_post['time_posted'] = job_soup.find(
                'span', {'class': 'posted-time-ago__text'}
            ).text.strip()
        except:
            job_post['time_posted'] = None

        try:
            job_post['num_applicants'] = job_soup.find(
                'span', {'class': 'num-applicants__caption'}
            ).text.strip()
        except:
            job_post['num_applicants'] = None
        
        job_criteria = {}
        for item in job_soup.find_all('li', class_='description__job-criteria-item'):
            try:
                label = item.find('h3', class_='description__job-criteria-subheader').text.strip()
                value = item.find('span', class_='description__job-criteria-text').text.strip()
                job_criteria[label] = value
            except:
                continue

        job_post['xp_level'] = job_criteria.get(list(job_criteria.keys())[0], None)
        job_post['job_type'] = job_criteria.get(list(job_criteria.keys())[1], None)
        job_post['job_sectors'] = job_criteria.get(list(job_criteria.keys())[3], None)

        try:
            job_post['job_description'] = job_soup.find(
                'div', {'class': 'show-more-less-html__markup'}
            ).get_text(separator="\n", strip=True)
        except:
            job_post['job_description'] = None

        job_list.append(job_post)

In [9]:
jobs_df = pd.DataFrame(job_list)
jobs_df

,job_id,job_title,company_name,work_model,location,time_posted,num_applicants,xp_level,job_type,job_sectors,job_description
0,4184262596,Analista de Operações de Serviços Financeiros ...,Contabilizei,Remoto,Brasil,Há 1 semana,None,Júnior,Tempo integral,Atividades dos serviços de tecnologia da infor...,A sua missão:\nApoiar para a eficiência operac...
1,4191196959,Estágio em Ciência de Dados,Bemobi,Remoto,Brasil,Há 7 horas,None,Não aplicável,Estágio,Atividades dos serviços de tecnologia da infor...,Na Bemobi valorizamos o espírito inovador e co...
2,4152258042,Estágio em Análise de Dados,E,Presencial,São Paulo e Região,Há 1 mês,159 candidaturas,Estágio,Tempo integral,Serviços advocatícios,Interpretar indicadores de performance ligados...
3,4179429367,Pessoa Desenvolvedora Backend Junior,Cora,Remoto,Brasil,Há 2 semanas,None,Assistente,Tempo integral,Atividades de serviços financeiros,Sua carreira com liberdade e propósito.\nVocê ...
4,4178256022,Assistente de Contas a Receber (Foco em Cobrança),Italac,Presencial,"São Paulo, SP",Há 2 semanas,98 candidaturas,Não aplicável,Tempo integral,Fabricação de produtos alimentícios e bebidas,Realizar trabalhos referentes a baixa no siste...
5,4184711760,Banco de Talentos - Analista de Políticas de C...,Neon,Remoto,Brasil,Há 1 semana,None,Assistente,Tempo integral,Atividades de serviços financeiros,"Sobre a Neon\nQueremos ser, cada vez mais, um ..."
6,4186563017,Analista Júnior | Gestão de Carteira de Crédito,XP Inc.,Presencial,"São Paulo, SP",Há 6 dias,None,Assistente,Tempo integral,Atividades de serviços financeiros,"Sobre nós\nNossa história começou em 2001, com..."
7,4192442079,Estagiário (a) de Dados,Lojas Eskala,Presencial,"São Paulo, SP",Há 1 dia,None,Estágio,Tempo integral,Comércio varejista,"DESCRIÇÃO\nSe você é apaixonado por dados, tem..."
8,4159081209,Analista de Sucesso do Cliente - Implementação,Omie,Remoto,Brasil,Há 1 mês,None,Não aplicável,Tempo integral,Atividades dos serviços de tecnologia da infor...,A Omie tem como propósito trazer prosperidade ...
9,4146772081,Estágio em Análise de Dados | Gente e Gestão C...,Solví Soluções para a Vida,Presencial,"São Paulo, SP",Há 1 mês,None,Não aplicável,Estágio,Fabricação de máquinas e equipamentos de uso i...,Estamos em busca de uma pessoa apaixonada por ...


In [2]:
work_model_id = 'random'
n_ids = 10
WORK_MODEL = {'1': 'Presencial', '2': 'Remoto', '3': 'Híbrido'}
JOB_KEYWORD = 'Dados'

if work_model_id not in ['1', '2', '3', 'random']:
    raise ValueError("Set an appropriate value for work_model_id ('1', '2', '3', 'random')")

job_data = {}

for n_results in range(0,n_ids,10):
    current_work_model = random.choice(list(WORK_MODEL.keys())) if work_model_id == 'random' else work_model_id
    
    list_url = f'https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords={JOB_KEYWORD}&location=Brasil&geoId=106057199&f_WT={\
        current_work_model}&position=1&start={n_results}'

    response = requests.get(list_url)
    list_soup = BeautifulSoup(response.text, 'html.parser')
    job_cards = list_soup.find_all('div', class_='base-card')

    for card in job_cards:
        if 'data-entity-urn' in card.attrs:
            job_id = card.get('data-entity-urn', '').split(':')[-1]

            if job_id and job_id not in job_data:
                job_data[job_id] = WORK_MODEL[current_work_model]

In [7]:
job_list = []
for job_id, job_work_model in job_data.items():
    job_url = f'https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{job_id}?_l=pt_BR'
    job_response = requests.get(job_url)

    if job_response.status_code == 200:
        job_post = {}
        job_soup = BeautifulSoup(job_response.text, 'html.parser')

        job_post['job_id'] = job_id

        try:
            job_post['job_title'] = job_soup.find(
                'h2', {'class': 'top-card-layout__title'}
            ).text.strip()
        except:
            job_post['job_title'] = None

        try:
            job_post['company_name'] = job_soup.find(
                'a', {'class': 'topcard__org-name-link'}
            ).text.strip()
        except:
            job_post['company_name'] = None

        job_post['work_model'] = job_work_model

        try:
            job_post['location'] = job_soup.find(
                'span', {'class': 'topcard__flavor topcard__flavor--bullet'}
            ).text.strip()
        except:
            job_post['location'] = None   

        try:
            job_post['time_posted'] = job_soup.find(
                'span', {'class': 'posted-time-ago__text'}
            ).text.strip()
        except:
            job_post['time_posted'] = None

        try:
            job_post['num_applicants'] = job_soup.find(
                'span', {'class': 'num-applicants__caption'}
            ).text.strip()
        except:
            job_post['num_applicants'] = None
        
        job_criteria = {}
        for item in job_soup.find_all('li', class_='description__job-criteria-item'):
            try:
                label = item.find('h3', class_='description__job-criteria-subheader').text.strip()
                value = item.find('span', class_='description__job-criteria-text').text.strip()
                job_criteria[label] = value
            except:
                continue

        job_post['xp_level'] = job_criteria.get(list(job_criteria.keys())[0], None)
        job_post['job_type'] = job_criteria.get(list(job_criteria.keys())[1], None)
        job_post['job_sectors'] = job_criteria.get(list(job_criteria.keys())[3], None)

        try:
            job_post['job_description'] = job_soup.find(
                'div', {'class': 'show-more-less-html__markup'}
            ).get_text(separator="\n", strip=True)
        except:
            job_post['job_description'] = None

        job_list.append(job_post)


In [3]:
df_jobs = pd.read_csv('../data/raw/jobs_data.csv', index_col=0)

In [8]:
df_jobs

,job_id,job_title,company_name,work_model,location,time_posted,num_applicants,xp_level,job_type,job_sectors,job_description
0,4150644206,Estágio Analise de Dados,EPTV,Híbrido,"Campinas, SP",Há 1 mês,NaN,Pleno-sênior,Tempo integral,Atividades de rádio e de televisão,Descrição\nEstamos em busca de um(a) estagiári...
1,4180219635,Trainee de Desenvolvimento - Matarazzo,RSM Brasil,Híbrido,"São Paulo, SP",Há 2 semanas,NaN,Não aplicável,Tempo integral,Atividades de contabilidade,Lugar de gente inovadora que faz acontecer!\nS...
2,4169347554,Analista Júnior de Produtos - Cartões,Banco Bmg,Híbrido,"São Paulo, SP",Há 6 dias,NaN,Não aplicável,Tempo integral,Bancos,"Há mais de 90 anos, o Banco Bmg vem construind..."
3,4168263642,ESTÁGIO EM BUSINESS INTELLIGENCE,Zilor,Híbrido,"São Paulo, SP",Há 4 semanas,NaN,Estágio,Estágio,Agricultura,Somos uma indústria brasileira com atuação glo...
4,4150286242,Analista Júnior de Dados - Prevenção à Fraude,Shopee,Híbrido,"São Paulo, SP",Há 2 semanas,NaN,Assistente,Tempo integral,Plataformas de comercialização na Internet e T...,"About The Team\nSobre a Shô\nAqui na Shô, acre..."
5,4075018778,Estágio Comercial REDE | Itaú Unibanco - VAGAS...,Itaú Unibanco,Híbrido,"São Paulo, SP",Há 5 dias,NaN,Assistente,Tempo integral,Bancos,Que tal construir a REDE com a gente? 🚀\nHá 26...
6,4149731885,Estágio em Sistemas da Informação,EPTV,Híbrido,"Campinas, SP",Há 1 mês,178 candidaturas,Assistente,Tempo integral,Atividades de rádio e de televisão,Descrição\nO estagiário(a) de Sistemas da Info...
7,4075025088,Faça sua Carreira de Engenharia de Processos n...,Itaú Unibanco,Híbrido,"São Paulo, SP",Há 2 semanas,NaN,Assistente,Tempo integral,Bancos,Vem construir no Itaú_\nComo maior banco priva...
8,4187888381,Estagiário de Análise de Dados,Neon Comercial Reagentes Analíticos,Híbrido,São Paulo e Região,Há 1 semana,NaN,Estágio,Estágio,Fabricação de produtos químicos,"A NEON, ciente de sua responsabilidade social ..."
9,4187544975,Analista de Dados e BI Júnior I Modelo Hibrido,Fast Shop S/A,Híbrido,"São Paulo, SP",Há 1 semana,NaN,Não aplicável,Tempo integral,Comércio varejista,Somos uma empresa brasileira de varejo reconhe...


---

In [ ]:
from urllib.parse import quote

# KEYWORDS = ['Data Analyst', 'Cientista de Dados', 'Engenheiro de Dados', 'Data Scientist',
#             'Data Engineer', 'Analista de Dados']
KEYWORDS = ['Data Analyst']

In [36]:
def format_keyword(keyword):
    return f'"{quote(str(keyword))}"'

work_type_counts = {}

for keyword_raw in KEYWORDS:
    keyword = format_keyword(keyword_raw)

    url = f'https://www.linkedin.com/jobs/search?keywords={keyword}&location=Brasil&geoId=106057199'

    print(url)

    response = requests.get(url)
    list_soup = BeautifulSoup(response.text, 'html.parser')
    work_type_filters = list_soup.find_all('div', class_='filter-values-container__filter-value')

    keyword_counts = {}
    
    # Loop through each filter value to extract the work type and count
    for filter_div in work_type_filters:
        label = filter_div.find('label')
        if label:
            text = label.get_text(strip=True)
            if 'On-site' in text:
                keyword_counts['1'] = int(text.split('(')[-1].split(')')[0])
            elif 'Remote' in text:
                keyword_counts['2'] = int(text.split('(')[-1].split(')')[0])
            elif 'Hybrid' in text:
                keyword_counts['3'] = int(text.split('(')[-1].split(')')[0])
    
    work_type_counts[keyword_raw] = keyword_counts

# Print the extracted values
work_type_counts


https://www.linkedin.com/jobs/search?keywords="Data%20Analyst"&location=Brasil&geoId=106057199


{'Data Analyst': {'2': 55, '1': 51, '3': 40}}

In [33]:
for keyword, wt_count in work_type_counts.items():
    for id, count in wt_count.items():
        print(keyword, id, count)

Data Analyst 2 55
Data Analyst 1 51
Data Analyst 3 40


In [44]:
job_ids_cache = {}
counter_new_ids = 0
for keyword, wt_count in work_type_counts.items():
    for id, count in wt_count.items():
        
        keyword = format_keyword(keyword)
        n_ids = count
        current_work_model = id

        
        
        for n_results in range(0, n_ids, 10):
            print(keyword, id, count)
            list_url = f'https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords={keyword}&location=Brasil&geoId=106057199&f_WT={\
                current_work_model}&position=1&start={n_results}'

            response = requests.get(url)
                
            list_soup = BeautifulSoup(response.text, 'html.parser')
            job_cards = list_soup.find_all('div', class_='base-card')

            for card in job_cards:
                job_id = card.get('data-entity-urn', '').split(':')[-1]
                if job_id and job_id not in job_ids_cache:
                    job_ids_cache[job_id] = WORK_MODEL[current_work_model]
                    counter_new_ids += 1

job_ids_cache

"Data%20Analyst" 2 55
"Data%20Analyst" 2 55
"Data%20Analyst" 2 55
"Data%20Analyst" 2 55
"Data%20Analyst" 2 55
"Data%20Analyst" 2 55
"%22Data%2520Analyst%22" 1 51
"%22Data%2520Analyst%22" 1 51
"%22Data%2520Analyst%22" 1 51
"%22Data%2520Analyst%22" 1 51
"%22Data%2520Analyst%22" 1 51
"%22Data%2520Analyst%22" 1 51
"%22%2522Data%252520Analyst%2522%22" 3 40
"%22%2522Data%252520Analyst%2522%22" 3 40
"%22%2522Data%252520Analyst%2522%22" 3 40
"%22%2522Data%252520Analyst%2522%22" 3 40


{'4184286787': 'Remoto',
 '4172358585': 'Remoto',
 '4187797503': 'Remoto',
 '4124164977': 'Remoto',
 '4186538629': 'Remoto',
 '4187566138': 'Remoto',
 '4190732442': 'Remoto',
 '4178559401': 'Remoto',
 '4186560108': 'Remoto',
 '4185986202': 'Remoto',
 '4192722817': 'Remoto',
 '3791113697': 'Remoto',
 '4196518023': 'Remoto',
 '4179409836': 'Remoto',
 '4178561257': 'Remoto',
 '4193624659': 'Remoto',
 '4191627456': 'Remoto',
 '4187796673': 'Remoto',
 '4185033111': 'Remoto',
 '4190161081': 'Remoto',
 '4195477470': 'Remoto',
 '4191513475': 'Remoto',
 '4181502604': 'Remoto',
 '4186292100': 'Remoto',
 '4195176363': 'Remoto',
 '4153114497': 'Remoto',
 '4180987902': 'Remoto',
 '4181299722': 'Remoto',
 '4183181863': 'Remoto',
 '4132557685': 'Remoto',
 '4137719160': 'Remoto',
 '4190730024': 'Remoto',
 '4178535739': 'Remoto',
 '4167015501': 'Remoto',
 '4187054380': 'Remoto',
 '4167847288': 'Remoto',
 '4165758485': 'Remoto',
 '4190446680': 'Remoto',
 '4180589625': 'Remoto',
 '4186294240': 'Remoto',


In [46]:
len(job_ids_cache)

60

---

In [12]:
jobs = {
    'Data Analyst': {2: 68, 1: 55, 3: 42}, 
    'Cientista de Dados': {2: 101, 1: 86, 3: 80}, 
    'Engenheiro de Dados': {1: 624, 2: 300, 3: 237}, 
    'Data Scientist': {2: 109, 1: 57, 3: 54}, 
    'Data Engineer': {2: 211, 1: 77, 3: 76}, 
    'Analista de Dados': {1: 360, 3: 176, 2: 83}
    }

jobs

{'Data Analyst': {2: 68, 1: 55, 3: 42},
 'Cientista de Dados': {2: 101, 1: 86, 3: 80},
 'Engenheiro de Dados': {1: 624, 2: 300, 3: 237},
 'Data Scientist': {2: 109, 1: 57, 3: 54},
 'Data Engineer': {2: 211, 1: 77, 3: 76},
 'Analista de Dados': {1: 360, 3: 176, 2: 83}}

In [24]:
num_keywords = len(jobs)
total_postings = sum(sum(job_counts.values()) for job_counts in jobs.values())

print(f"Successfully fetched job data for {num_keywords} keywords and {total_postings} job postings.")


print(f"Successfully fetched job data for {len(jobs)} keywords.")

Successfully fetched job data for 6 keywords and 2796 job postings.
Successfully fetched job data for 6 keywords.


In [ ]:
get_job_ids(self, n_ids, keyword_raw=JOB_KEYWORD, work_model_id='random'):

In [10]:
jobs.items()

dict_items([('Data Analyst', {2: 68, 1: 55, 3: 42}), ('Cientista de Dados', {2: 101, 1: 86, 3: 80}), ('Engenheiro de Dados', {1: 624, 2: 300, 3: 237}), ('Data Scientist', {2: 109, 1: 57, 3: 54}), ('Data Engineer', {2: 211, 1: 77, 3: 76}), ('Analista de Dados', {1: 360, 3: 176, 2: 83})])

In [5]:
soma = 0
for position, data in jobs.items():
    for work_model_id, n_ids in data.items():
        soma += n_ids
print(soma)

2796


In [11]:
df = pd.read_csv('../data/raw/jobs_data.csv')
df

,job_id,work_model,job_title,company_name,location,time_posted,num_applicants,xp_level,job_type,job_sectors,job_description
0,4190732442,Remoto,Data Analyst,Dandy,Brasil,Há 1 dia,88 candidaturas,Assistente,Tempo integral,Fabricação de instrumentos e materiais para us...,Dandy is transforming the massive ($200B) but ...
1,4172358585,Remoto,Data Analyst,Onebridge,Brasil,Há 1 semana,NaN,Pleno-sênior,Tempo integral,Atividades dos serviços de tecnologia da infor...,Onebridge is a Consulting firm with an HQ in I...
2,4186560108,Remoto,Data Analyst,HCLTech,Brasil,Há 3 semanas,NaN,Pleno-sênior,Tempo integral,"Tecnologia, Informação e Mídia","HCLTech is a global technology company, spread..."
3,4191627456,Remoto,Data Analyst,"CloudWalk, Inc.","São Paulo, SP",Há 2 semanas,88 candidaturas,Não aplicável,Tempo integral,Tecnologia da informação e serviços,About CloudWalk:\nWe are not just another fint...
4,4200988702,Remoto,Data Analyst,iVisa,Brasil,Há 1 semana,NaN,Pleno-sênior,Tempo integral,Tecnologia da informação e serviços,We’re growing! Don't miss the opportunity to b...
...,...,...,...,...,...,...,...,...,...,...,...
2500,4200650306,Híbrido,Customer Technology Adivisor ( Pre-sale & Solu...,Kyndryl,São Paulo e Região,Há 3 dias,NaN,Assistente,Tempo integral,Atividades dos serviços de tecnologia da infor...,"Who We Are\nAt Kyndryl, we design, build, mana..."
2501,4207529566,Remoto,Blockchain Data Engineer (Senior/Lead) ID34521,AgileEngine,Porto Alegre e Região,Há 15 horas,NaN,Pleno-sênior,Tempo integral,Atividades dos serviços de tecnologia da infor...,AgileEngine is one of the Inc. 5000 fastest-gr...
2502,4208757675,Híbrido,Cientista de dados bi,Netvagas,"Nova Lima, MG",Há 1 dia,NaN,Assistente,Tempo integral,Fornecimento e gestão de recursos humanos,A Kalendae atua desde 2005 no mercado brasilei...
2503,4206900989,Presencial,Analista de Dados Cloud,Zeentech,"Camaçari, BA",Há 3 horas,NaN,Assistente,Tempo integral,Atividades de consultoria em gestão empresarial,"Cidade\nCamaçari, BA, Brasil\nDescrição da Vag..."
